# Feasibility modeling



A more complicated example involves minimizing in two dimensions, where some parts of the parameter space are off-limits. Let's minimize Himmelblau's function, subject to the constraint that $x_1^2 + x_2^2 < 50$

In [ ]:
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from bloptools import test_functions

x1 = x2 = np.linspace(-8, 8, 256)
X1, X2 = np.meshgrid(x1, x2)
from bloptools.tasks import Task

task = Task(key="himmelblau", kind="min")
F = test_functions.constrained_himmelblau(X1, X2)

plt.pcolormesh(x1, x2, F, norm=mpl.colors.LogNorm(), shading="auto")
plt.colorbar()
plt.xlabel("x1")
plt.ylabel("x2")

where everything outside our constraint is undefined. In our digestion function, we return a `NaN` when we violate the constraint:

In [ ]:
def digestion(db, uid):
    products = db[uid].table()

    for index, entry in products.iterrows():
        products.loc[index, "himmelblau"] = test_functions.constrained_himmelblau(entry.x1, entry.x2)

    return products

and create the agent in the usual way:

In [ ]:
from bloptools.utils import prepare_re_env

%run -i $prepare_re_env.__file__ --db-type=temp

from bloptools import devices
from bloptools.bayesian import Agent

dofs = [
    {"device": devices.DOF(name="x1"), "limits": (-8, 8), "kind": "active"},
    {"device": devices.DOF(name="x2"), "limits": (-8, 8), "kind": "active"},
]

tasks = [
    {"key": "himmelblau", "kind": "minimize"},
]

agent = Agent(
    dofs=dofs,
    tasks=tasks,
    digestion=digestion,
    db=db,
)

RE(agent.initialize("qr", n_init=64))

agent.plot_tasks()

In addition to modeling the fitness of the task, the agent models the probability that an input will be feasible:

In [ ]:
agent.plot_validity()

It combines the estimate of the objective and the estimate of the feasibility in deciding where to go:

In [ ]:
agent.plot_acquisition(acq_func=["ei", "pi", "ucb"])

In [ ]:
RE(agent.learn("ei", n_per_iter=4))

The agent automatically tries to avoid infeasible points, but will end up naturally exploring the boundary of the constraint. Let's see where the agent is thinking of going:

In [ ]:
agent.plot_tasks()
agent.plot_acquisition(strategy=["ei", "pi", "ucb"])

The agent will naturally explore the whole parameter space

In [ ]:
RE(agent.learn("ei", n_iter=16))
agent.plot_tasks()